In [13]:
import numpy as np 
import pandas as pd 
from prophet import Prophet
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import model_selection
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.preprocessing import PolynomialFeatures
from statsmodels.tsa.stattools import adfuller
!pip install pyramid-arima
from pmdarima.arima import auto_arima
std=StandardScaler()
import plotly.graph_objects as go


#pd.set_option('display.float_format', lambda x: '%.6f' % x)

  Using cached pyramid-arima-0.8.1.tar.gz (515 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pyramid-arima
Failed to build pyramid-arima
  Running setup.py install for pyramid-arima: started
  Running setup.py install for pyramid-arima: finished with status 'error'

[notice] A new release of pip available: 22.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [186 lines of output]
  Partial import of pyramid during the build process.
  
  Requirements: ['Cython>=0.23\nnumpy>=1.10\nscipy>=0.9\nscikit-learn>=0.17\npandas>=0.19\nstatsmodels>=0.9.0\n']
  Adding extra setuptools args
  C:\Users\anure\AppData\Local\Temp\pip-install-j729bkij\pyramid-arima_dca25b671fe54b1fac6c096b9ef3e3fb\setup.py:225: DeprecationWarning:
  
    `numpy.distutils` is deprecated since NumPy 1.23.0, as a result
    of the deprecation of `distutils` itself. It will be removed for
    Python >= 3.12. For older Python versions it will remain present.
    It is recommended to use `setuptools < 60.0` for those Python versions.
    For more details, see:
      https://numpy.org/devdocs/reference/distutils_status_migration.html
  
  
    from numpy.distutils.core import setup
  C:\Users\anure\AppData\Local\Programs\Python\Python310\lib\site-packages\setuptools

In [14]:
Dataset = pd.read_csv('number-of-reported-smallpox-cases.csv')
# Dataset.set_index('Year', inplace=True)
# Dataset['Year'] = pd.to_datetime(Dataset['Year'], format='%Y')

Dataset.drop(['Entity', 'Code'],axis=1,inplace=True)
Dataset.rename(columns = {'Year':'ds', 'Reported Smallpox cases (WHO)':'y'}, inplace = True)
Dataset.dtypes
# cols=["Year","Month","Day"]
# Dataset['Date'] = Dataset[cols].apply(lambda x: '-'.join(x.values.astype(str)), axis="columns")
Dataset

,ds,y
0,1949,393
1,1950,612
2,1951,1299
3,1952,2179
4,1953,1813
...,...,...
4148,1973,0
4149,1974,0
4150,1975,0
4151,1976,0


In [15]:
fig = px.line(Dataset, x="ds", y="y")
fig.show()

In [16]:
Dataset=Dataset.query("ds< 1979")

Dataset.dtypes

ds    int64
y     int64
dtype: object

In [17]:
model_compare=Dataset.iloc[45:]
error_compare = pd.DataFrame({'Model': pd.Series(dtype='str'),
                   'RMSE': pd.Series(dtype='float64')})
error_compare

,Model,RMSE


# Linear Regression

In [7]:
train_ml=Dataset.iloc[:45]
valid_ml=Dataset.iloc[45:]
model_scores=[]

In [8]:
lin_reg=LinearRegression(normalize=True)

In [ ]:
lin_reg.fit(np.array(train_ml["ds"]).reshape(-1,1),np.array(train_ml["y"]).reshape(-1,1))

In [4]:
prediction_valid_linreg=lin_reg.predict(np.array(valid_ml["ds"]).reshape(-1,1))

NameError: name 'lin_reg' is not defined

In [ ]:
model_scores.append(np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg)))
print("Root Mean Square Error for Linear Regression: ",np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg)))

In [5]:
model_compare["Linear Regression"]= prediction_valid_linreg
model_compare
# prediction_valid_linreg

NameError: name 'prediction_valid_linreg' is not defined

In [ ]:
new_row = {'Model':'Linear Regression', 'RMSE': np.sqrt(mean_squared_error(valid_ml["y"],prediction_valid_linreg))}
#append row to the dataframe
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare



In [6]:

plt.figure(figsize=(11,6))
prediction_linreg=lin_reg.predict(np.array(Dataset["ds"]).reshape(-1,1))
linreg_output=[]
for i in range(prediction_linreg.shape[0]):
    linreg_output.append(prediction_linreg[i][0])

fig=go.Figure()
fig.add_trace(go.Scatter(x=Dataset["ds"], y=Dataset["y"],
                    mode='lines+markers',name="Train Data for Anomaly"))
fig.add_trace(go.Scatter(x=Dataset["ds"], y=linreg_output,
                    mode='lines',name="Linear Regression Best Fit Line",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Linear Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

NameError: name 'lin_reg' is not defined

<Figure size 792x432 with 0 Axes>

# Polynomial Regression

In [ ]:
train_ml=Dataset.iloc[:45]
valid_ml=Dataset.iloc[45:]

In [7]:
poly = PolynomialFeatures(degree = 4) 

In [8]:
train_poly=poly.fit_transform(np.array(train_ml["ds"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(valid_ml["ds"]).reshape(-1,1))
y=train_ml["y"]

NameError: name 'train_ml' is not defined

In [ ]:
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y)

In [9]:
prediction_poly=linreg.predict(valid_poly)
rmse_poly=np.sqrt(mean_squared_error(valid_ml["y"],prediction_poly))
model_scores.append(rmse_poly)
print("Root Mean Squared Error for Polynomial Regression: ",rmse_poly)

NameError: name 'linreg' is not defined

In [10]:
new_row = {'Model':'Polynomial Regression', 'RMSE': model_scores[1]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare


NameError: name 'model_scores' is not defined

In [ ]:
comp_data=poly.fit_transform(np.array(Dataset["ds"]).reshape(-1,1))
plt.figure(figsize=(11,6))
predictions_poly=linreg.predict(comp_data)

fig=go.Figure()
fig.add_trace(go.Scatter(x=Dataset["ds"], y=Dataset["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=Dataset["ds"], y=predictions_poly,
                    mode='lines',name="Polynomial Regression Best Fit",
                    line=dict(color='black', dash='dot')))
fig.update_layout(title="Confirmed Cases Polynomial Regression Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",
                 legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [11]:
Dataset.dtypes

ds                                                                    int64
Adults (ages 15+) and children (ages 0-14) newly infected with HIV    int64
dtype: object

In [12]:
new_prediction_poly=[]
for i in range(1,18):
    new_date_poly=poly.fit_transform(np.array(Dataset["ds"].max()+i).reshape(-1,1))
    new_prediction_poly.append(linreg.predict(new_date_poly)[0])

NameError: name 'linreg' is not defined

# SVM

In [ ]:
# svm=SVR(C=1,degree=6,kernel='poly',epsilon=0.01)

In [13]:
#Fitting model on the training data
# svm.fit(np.array(train_ml["Year"]).reshape(-1,1),np.array(train_ml["Anomaly"]).reshape(-1,1))

In [14]:

# prediction_valid_svm=svm.predict(np.array(valid_ml["Year"]).reshape(-1,1))

In [15]:
Dataset.dtypes

Dataset.dtypes




# new_date=[]
# new_prediction_lr=[]
# new_prediction_svm=[]
# for i in range(1,18):
#     new_date.append(Dataset.index[-1]+timedelta(year=i))
#     new_prediction_lr.append(lin_reg.predict(np.array(Dataset.index.max()+i).reshape(-1,1))[0][0])
# #     new_prediction_svm.append(svm.predict(np.array(datewise["Days Since"].max()+i).reshape(-1,1))[0])



# pd.set_option('display.float_format', lambda x: '%.6f' % x)
# model_predictions=pd.DataFrame(zip(new_date,new_prediction_lr,new_prediction_poly),
#                                columns=["Dates","Linear Regression Prediction","Polynonmial Regression Prediction"])
# model_predictions.head()

ds                                                                    int64
Adults (ages 15+) and children (ages 0-14) newly infected with HIV    int64
dtype: object

# Time Series Forecasting 

## Holt's Linear Model

In [16]:
model_train=Dataset.iloc[:45]
valid=Dataset.iloc[45:]
y_pred=valid.copy()
valid

,ds,Adults (ages 15+) and children (ages 0-14) newly infected with HIV
45,2004,100
46,2005,100
47,2006,100
48,2007,100
49,2008,100
...,...,...
4064,2016,41000
4065,2017,36000
4066,2018,30000
4067,2019,30000


In [17]:
holt=Holt(np.asarray(model_train["y"])).fit(optimized=True)   

KeyError: 'y'

In [ ]:
y_pred["Holt"]=holt.forecast(len(valid))
model_scores.append(np.sqrt(mean_squared_error(y_pred["y"],y_pred["Holt"])))
print("Root Mean Square Error Holt's Linear Model: ",np.sqrt(mean_squared_error(y_pred["y"],y_pred["Holt"])))

In [ ]:
model_compare["Holt's Linear Model"]= y_pred["Holt"]
model_compare


In [ ]:
new_row = {'Model':'Holt\'s Linear Model', 'RMSE': model_scores[2]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=y_pred["Holt"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases Holt's Linear Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

# AR Model (using AUTO ARIMA)

In [ ]:
model_train=Dataset.iloc[:45]
valid=Dataset.iloc[45:]
y_pred=valid.copy()
valid

In [ ]:
model_ar= auto_arima(model_train["y"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["y"])

In [ ]:
def ad_test(dataset):
    dftest=adfuller(dataset,autolag='AIC')
    print("1. ADF: ",dftest[0])
    print("2. P-Value: ",dftest[1])
    print("3. No. of Lags: ",dftest[2])
    print("4. Num of Observations Used for ADF Regression and Critical Value Calculation: ",dftest[3])
    print("5. Critical Values: ")
    for key, val in dftest[4].items():
        print("\t", key, ": ",val)

In [ ]:
ad_test(model_train['y'])

In [ ]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar

In [ ]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["y"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["y"],y_pred["AR Model Prediction"])))

In [ ]:
model_compare["AR Model Prediction"]= y_pred["AR Model Prediction"]
model_compare



In [ ]:
new_row = {'Model': 'AR Model Prediction', 'RMSE': model_scores[3]}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of Confirmed Cases",))
fig.update_layout(title="Confirmed Cases AR Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
model_predictions = pd.DataFrame()
AR_model_new_prediction=[]
for i in range(1,18):
    AR_model_new_prediction.append(model_ar.predict(len(valid)+i)[-1])
model_predictions["AR Model Prediction"]=AR_model_new_prediction
model_predictions

# Fb-Prophet

In [ ]:
model_train=Dataset.iloc[:45]
valid=Dataset.iloc[45:]
model_train.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)
valid.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)
# model_train

fb_prophet = Prophet(interval_width = 0.95, yearly_seasonality = True)
fb_prophet.fit(model_train)

In [ ]:
in_forecast = fb_prophet.predict(valid)
in_forecast['ds'] = in_forecast['ds'].apply(lambda x: x.year)
in_forecast

In [ ]:
Dataset_prophet = Dataset.copy()

Dataset_prophet.rename(columns = {'Year':'ds', 'Anomaly':'y'}, inplace = True)



in_result_df = Dataset_prophet.merge(in_forecast, on='ds', how='left')

fig = px.line(
    in_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

In [ ]:
future_df = pd.DataFrame(np.arange(2022, 2031), columns=['ds'])
future_df

In [ ]:
out_model = Prophet(interval_width = 0.95, yearly_seasonality = True)
out_model.fit(Dataset_prophet)

In [ ]:
out_forecast = out_model.predict(future_df)
out_forecast['ds'] = out_forecast['ds'].apply(lambda x: x.year)
out_forecast

In [ ]:
import plotly.express as px
out_result_df = pd.concat([Dataset_prophet, out_forecast], axis=0)
fig = px.line(
    out_result_df,
    x='ds',
    y=['y', 'yhat'],
    color_discrete_sequence=['black', 'red'],
    labels={'ds': "Year"},
    title="Average Global Temperature Change Over Time"
)

fig.show()

In [ ]:
tempdf=in_forecast[['ds','yhat']]

tempdf.rename(columns = {'ds':'ds', 'yhat':'FB-Prophet Prediction'}, inplace = True)



model_compare=model_compare.merge(tempdf)

# model_compare.rename(columns = {'yhat':'FB-Prophet Prediction'}, inplace = True)

# model_compare.head()

model_compare

# model_compare["AR Model Prediction"]= y_pred["AR Model Prediction"]
# model_compare

In [ ]:
new_row2 = {'Model':'FB-Prophet', 'RMSE': np.sqrt(mean_squared_error(model_compare["y"],model_compare["FB-Prophet Prediction"]))}
#append row to the dataframe
error_compare = error_compare.append(new_row2, ignore_index=True)
error_compare

# SARIMA Model (using AUTO ARIMA)

In [ ]:
Train = Dataset.iloc[:45]
Valid = Dataset.iloc[45:]
Y_pred_sarima = valid.copy()

In [ ]:
model_sarima= auto_arima(Train["y"],trace=True, error_action='ignore', 
                         start_p=0,start_q=0,max_p=2,max_q=2,m=7,
                   suppress_warnings=True,stepwise=True,seasonal=True)
model_sarima.fit(Train["y"])

In [ ]:
prediction_sarima=model_sarima.predict(len(Valid))
Y_pred_sarima["SARIMA Model Prediction"]=prediction_sarima

In [ ]:
# model_scores.append(np.sqrt(mean_squared_error(Y_pred_sarima["Anomaly"],Y_pred_sarima["SARIMA Model Prediction"])))
# print("Root Mean Square Error for SARIMA Model: ",np.sqrt(mean_squared_error(Y_pred_sarima["Anomaly"],Y_pred_sarima["SARIMA Model Prediction"])))





In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train["ds"], y=model_train["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))
fig.add_trace(go.Scatter(x=valid["ds"], y=valid["y"],
                    mode='lines+markers',name="Validation Data for Confirmed Cases",))
fig.add_trace(go.Scatter(x=valid["ds"], y=Y_pred_sarima["SARIMA Model Prediction"],
                    mode='lines+markers',name="Prediction for Confirmed Cases",))
fig.update_layout(title="Confirmed Cases SARIMA Model Prediction",
                 xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

In [ ]:
model_compare["SARIMA"] = Y_pred_sarima["SARIMA Model Prediction"]
model_compare


In [ ]:
new_row = {'Model': 'SARIMA Model Prediction', 'RMSE': np.sqrt(mean_squared_error(y_pred["Anomaly"],y_pred["SARIMA Model Prediction"]))}
error_compare = error_compare.append(new_row, ignore_index=True)
error_compare
# 